#Load the dataset:

In [79]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("camel-ai/physics", split="train")




#Prepare the SmolLM2-360M model:

In [81]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "HuggingFaceTB/SmolLM2-360M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [82]:
# Define padding token if necessary
tokenizer.pad_token = tokenizer.eos_token or tokenizer.bos_token or "[PAD]"


#Preprocess the dataset:

In [83]:
# Preprocess function
def preprocess_function(examples):
    # Concatenate the problem and solution for training a causal language model
    inputs = [f"Problem: {problem}\nSolution: {solution}" for problem, solution in zip(examples["message_1"], examples["message_2"])]
    
    # Tokenize inputs
    model_inputs = tokenizer(inputs, truncation=True, padding="max_length", max_length=512)
    
    # Set labels to be the same as input_ids, shifting required for causal language modeling
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    
    return model_inputs

In [84]:
# Tokenize and preprocess the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)


In [ ]:
tokenized_dataset

In [86]:
# Using 80% for training and 20% for validation
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)


In [87]:
# import torch
# torch.mps.empty_cache()


In [88]:
import numpy as np
# 5. Define Evaluation Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # Compare predictions and labels
    accuracy = (predictions == labels).mean()
    return {"accuracy": accuracy}

#Set up the training arguments:

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    logging_steps=100,
    save_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    gradient_checkpointing=True,
    push_to_hub=True,
    hub_model_id="fine-tuned-smolLM2-360M-on-physics",
)

#Create a Trainer and start fine-tuning:

In [ ]:
import torch

# Clear cache
torch.cuda.empty_cache()

# Reset memory allocations and free up GPU memory
torch.cuda.memory_summary(device=None, abbreviated=False)


In [ ]:
import numpy as np
from transformers import Trainer

trainer = Trainer(
    
    model=model,
    args=training_args,
    train_dataset=train_test_split["train"],
    eval_dataset=train_test_split["test"],
    compute_metrics=compute_metrics, 
)

trainer.train()



#Save the fine-tuned model:

In [ ]:
model.save_pretrained("./fine-tuned-smolLM2-360M-on-physics")
tokenizer.save_pretrained("./fine-tuned-smolLM2-360M-on-physics")
# trainer.push_to_hub()
trainer.push_to_hub(commit_message="Fine-tuned smolLM2-360M on camel-ai/physics")


In [ ]:
from huggingface_hub import HfApi

api = HfApi()
api.upload_folder(
    folder_path="fine-tuned-smolLM2-360M-on-physics",
    repo_id="akhilfau/fine-tuned-smolLM2-360M-on-physics"
)


In [ ]:
#testing

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "akhilfau/fine-tuned-smolLM2-360M-on-physics"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Test with a sample input
input_text = "Question: What is gravity?"
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(inputs["input_ids"], max_length=50)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))
